# RedBull.TV Film Downloader

## Overview
RedBull.TV runs Adobe Media Server. It presents its content as HLS, which is file container format optimized for Apple devices. The container supports H.264/AAC-encoded video/audio and is based on MPEG-2 transport stream (TS). All video delivered to iOS (including AIR for IOS) must use this format.

I really wanted to watch "Two Fast", https://www.redbull.tv/film/AP-1PYF79CXW1W11/two-fast . The film is mate of multiple TS chunks, each 3-second long.


## Getting playlist.m3u8

Using Chrome or alternative browser, navigate to the film's landing page, select desired resolution (i.e. 1920x1080). Enable Developer Console. Reload the page, sort the "Network" list by filename, and find the needed file:

<img src="finding_playlist.png">


## Downloading with PyCurl

In [321]:
import os
import re
import pycurl

In [322]:
filename = 'playlist.m3u8'
pattern = '..ts|[0-9]..ts|[0-9][0-9]..ts'

That pattern is up to 3-digits in the filename. If the film is longer, you may need to add another set.
Reference is https://docs.python.org/2/library/re.html
If you have a more elegant solution, let me know.

In [323]:
with open(filename, 'r') as f:
    lines = f.readlines()


In [326]:
#for tests, try with last lines ones only 
#for line in lines[-6:]:
for line in lines:
    match = re.search(pattern, line)
    if match:
        #newfile.append(match.group(0))
        #cleanurl.append(line.rstrip("\n"))
        ts = open(match.group(0), "wb")
        curl = pycurl.Curl()
        curl.setopt(pycurl.URL, str(line.rstrip("\n")))
        curl.setopt(pycurl.WRITEDATA, ts)
        curl.perform()
        curl.close()
        ts.close()        

## What to do with Bunch of .ts files 

Concatenate them, and use ffmpeg if you prefer to encode them again: https://superuser.com/questions/692990/use-ffmpeg-copy-codec-to-combine-ts-files-into-a-single-mp4 , i.e. for Mac OS X it's:

    cat $(ls *.ts | sort -n) > movie.ts

